## Data cleaning

in this section, the dataset will be cleaned and restricted to have only the neceaasry values to predict if a game will be a hit or a flop.


check if there are NAN values in the dataset:


In [ ]:
import pandas as pd
import numpy as np

df = pd.read_csv("vgsales.csv")
df.isna().sum()


### How to Handle NaN values in this project

Publisher NAN values\
Since there are 16958 data entries, 58 NaN values correspond ± to the 0.4% of the dataset.
Given this small value, the best soluton is to remove these columns from teh set.

Year NaN values\
The number of entries that is missing is much larger than the number of nan values for the publisher, but still not very large, so, the best solution is to infer those values from the other one in the dataset.
The values can be calculated with the mean or the median, which one to choode depends by the dataset, if it has a small standard deviation, with nit much outliers, probalby the mean will be good. But if thre is a high standard deviation of if there are multiple big outliers, median will be better because less affected. 



below we are going to remove the values of publisher.

In [ ]:
df_cleaned = df.dropna(subset=["Publisher"])
df_cleaned.isna().sum()

below we are going to analyse standard deviation and outliers to decide what approach to use for calculating the missing years.

In [ ]:
#removing nan values to calculate standard deviation
arr_of_all_years = df_cleaned["Year"]
arr_of_all_years = arr_of_all_years[np.isnan(arr_of_all_years) == False]
arr_of_all_years.isna().sum()
standard_deviation = np.std(arr_of_all_years)
print(f"Standard deviation = {standard_deviation}")

#use z-score to find outliers
df_cleaned["z_score"] = (df_cleaned["Year"] - df_cleaned["Year"].mean()) / df_cleaned["Year"].std()
print(f"Number of outliers: {(np.abs(df_cleaned["z_score"]) > 3).sum()}")


As we can see from the results, both standard deviation and ouliers are small, so we can calcualte the missing years with the mean of the present Year value.

In [ ]:
df_cleaned.fillna(df_cleaned["Year"].mean(), inplace=True)
df_cleaned.isna().sum()

### Removing unnecessary data 

Not all the columns are necessary to train the model to recognize if a game will be a hit or a flop, too much values can cause overfitting.\
Our team decided to keep only some of the columns, more precisely: Platform, Year, Genre, Publisher, Global Sales

In [ ]:
#filter for only the desired columns
columns = ['Platform', 'Year', 'Genre', 'Publisher', 'Global_Sales']
df_filtered = df_cleaned[columns].copy()

print(df_filtered["Platform"].describe())
print(df_filtered["Genre"].describe())
print(df_filtered["Publisher"].describe())
df_filtered.head(15)


### Platform column cleaning 

In the next step, data encoding, to guarantee that a platform is not considered mathematically greater than another, we are going to use one-hot encoding, that will create new columns.\
To reduce the number of generated columns and reduce overfitting, we are going to group platforms that have a few number of games (that will likely not heavily influence the decisions of the model due to the low number of entries) in a single category called other_platforms.

In [ ]:
keys = (df_filtered["Platform"].value_counts().keys().to_list())
values = (df_filtered["Platform"].value_counts().values.tolist())
platforms = df_filtered["Platform"].to_list()
print(platforms)

#group platforms that have both published less than 60 games and are also deprecated
x = 0
for i in platforms:
    index = keys.index(i)

    if (values[index] <= 60):
        platforms[x] = "Other_platform"

    x += 1

df_filtered["Platform"] = platforms

#as we can see from the output, we reduce the number of platform from 31 to 23
print(df_filtered["Platform"].describe())
df_filtered.head()

### Adding Is_hit column

Adding the Is_hit column is necessary for applying supervised learning concepts and enabling the model to identify patterns in the data.

We’ll use global_sales and platform as criteria.  A game is considered a hit if it’s in the 90th percentile of its platform sales. This approach avoids biases where only large platforms have hits while smaller platforms, even if their games were actually hits due to their size, are considered flops.


In [ ]:
df_temp = df_filtered.copy()

platform_90th_percentile = df_temp.groupby('Platform')['Global_Sales'].quantile(0.90)

df_temp = df_temp.merge(
    platform_90th_percentile.rename('Platform_Threshold'),
    on='Platform',
    how='left'
)

# replace inplace fillna with assignment to avoid chained-assignment warning

df_filtered["Is_hit"] = (df_temp['Global_Sales'] > df_temp['Platform_Threshold']).astype('Int64')
df_filtered["Is_hit"].fillna(0, inplace=True)
df_filtered.head(-10)

### Saving the filtered dataframe for visualizations

To do visual analysis we will save the latest non-encoded dataframe

In [12]:
df_filtered.to_csv("filtered_dataset.csv", index=False)

### Data encoding 

To make the data really understable by the model, we need to encode them in numerical values and then apply normalization techniques to standardise the values. 

For the Genre and the Platform columns, we can use One-Hot encoding, due to the fact that they have a low cardinality (both are below 35), we use One-Hot encoding to guarantee that the model will consider all the unordered data with the same importance.

For the publisher, that has more than 500 entries, we use label encoder to avpid getting too much columns and overfit the model.

In [ ]:
from sklearn.preprocessing import LabelEncoder

#encode the data
df_encoded = pd.get_dummies(df_filtered, columns=["Genre", "Platform"], drop_first=True, dtype=int)

#map the values from bool to int

label_encoder = LabelEncoder()
df_encoded["Publisher"] = label_encoder.fit_transform(df_encoded["Publisher"])

df_encoded.head(10)


### Normalize the Data

Once we obtain the numerical value of the strings, we need to apply normalization concepts to convert this data into a ragne that helps in preveting the model to consider more important values with higher numbers.\
Values created with one-hot encoding does not get Normalized since they are used only to say true/false.

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

#select the data to normalize
data_to_normalize = ["Year", "Publisher", "Global_Sales"]

df_encoded[data_to_normalize] = scaler.fit_transform(df_encoded[data_to_normalize])

df_encoded.head()

### Save the new .csv

In [ ]:
df_encoded.to_csv("cleaned_dataset.csv", index=False)